In [1]:
from tvDatafeed import TvDatafeed, Interval

In [14]:
# CONSTANTS
timeframes = {
  '4h': {
    'interval': Interval.in_4_hour,
    'table': 'ticks_4h',
    'bars': 4 * 4 * 365
  },
  # '1d': {
  #   'interval': Interval.in_daily,
  #   'table': 'ticks_1d',
  #   'bars': 5 * 365
  # },
  # '1w': {
  #   'interval': Interval.in_weekly,
  #   'table': 'ticks_1w',
  #   'bars': (8 * 365) / 5
  # }
}

# ASSET
symbol    = 'DIS'
exchange  = 'NYSE'

In [3]:
from helpers_db import get_engine, run_sql

engine = get_engine()

def fetch_asset(conn, symbol):
  return run_sql(conn, f"""SELECT * FROM assets WHERE symbol = '{symbol}' """).mappings().first()

def insert_asset(conn, symbol, exchange):
  run_sql(conn, f"""INSERT INTO assets (symbol, exchange, name) values ('{symbol}', '{exchange}', '{symbol}') """)
  
def fix_symbol(eachange_symbol):
  return eachange_symbol.split(':')[-1] if ':' in eachange_symbol else eachange_symbol

def normalize_dataframe(df):
  df.reset_index(inplace=True)
  df['symbol'] = df['symbol'].apply(fix_symbol)
  if 'datetime' in list(df.columns.values):
    df['dt'] = df['datetime']
    df.drop(columns=['datetime'], inplace=True)
  return df

def fetch_ticks(tv, symbol, exchange, interval=Interval.in_daily, bars=100, extended_session=True):
  df = tv.get_hist(
      symbol=symbol,
      exchange=exchange,
      interval=interval,
      n_bars=bars,
      extended_session=extended_session,
  )
  return normalize_dataframe(df)

# Check if the asset exists in KaizenBrain

In [4]:
with engine.begin() as conn:
  asset = fetch_asset(conn, symbol)

  # if not asset:
  #   insert_asset(conn, symbol, exchange)
  #   asset = fetch_asset(conn, symbol)
  
  print(asset)

{'symbol': 'DIS', 'name': 'DIS', 'exchange': 'NYSE', 'category': 'stock'}


In [13]:
print(timeframes)

{'4h': {'interval': <Interval.in_4_hour: '4H'>, 'table': 'ticks_4h', 'bars': 5840}, '1d': {'interval': <Interval.in_daily: '1D'>, 'table': 'ticks_1d', 'bars': 1825}, '1w': {'interval': <Interval.in_weekly: '1W'>, 'table': 'ticks_1w', 'bars': 584.0}}


# Fetch 4h, 1D and 1W ticks

In [22]:
from helpers_db import insert_on_conflict_nothing_ticks
from datetime import timezone, datetime
import pytz

tv = TvDatafeed()
my_timezone = pytz.timezone('Europe/Dublin')
with engine.begin() as conn:
  for key in timeframes.keys():
    timeframe = timeframes[key]
    bars      = 4 # int(timeframe['bars'])
    interval  = timeframe['interval']
    table     = timeframe['table']
    print(f"Fetching:: {symbol}:{exchange} {interval} {bars}")
    
    df = fetch_ticks(tv, symbol=symbol, exchange=exchange, bars=bars, interval=interval) # 7*24
    df['dt'] = df['dt'].dt.tz_localize(my_timezone)
    df['dt'] = df['dt'].dt.tz_convert(timezone.utc)
    print(df)

    # print(f"Inserting:: {symbol}:{exchange} into {table}")
    # df.to_sql(table, engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)
  # df.to_sql('ticks_1w', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)

Fetching:: DIS:NYSE Interval.in_4_hour 4
  symbol    open    high     low   close    volume                        dt
0    DIS  114.09  114.09  113.50  113.50    5503.0 2024-04-12 20:00:00+00:00
1    DIS  114.03  114.57  113.50  114.46    5437.0 2024-04-15 08:00:00+00:00
2    DIS  114.41  115.55  113.75  114.39  423632.0 2024-04-15 12:00:00+00:00
3    DIS  114.38  114.58  112.56  113.06  297767.0 2024-04-15 16:00:00+00:00
